In [81]:
import pandas as pd

In [82]:
df_1 = pd.read_csv('bestpoolshop_chlorine.csv')
df_1.head()

,title,overview,price,in_store_price,stock,description,link,product_id,calcium hypochlorite,other ingredients,trichlor-s-triazinetrione,available chlorine,active ingredient,manufacturer sku
0,22421 Poolife NST Non Stabilized Swimming Pool...,Use only in pools with a skimmer and skimmer b...,$185.76,$171.99,Only 7 left in stock,Poolife NST Non Stabilized Without Conditioner...,https://bestpoolshop.com/product/poolife-nst-n...,36140,70.2%,29.8%,NaN,NaN,NaN,22421
1,22422 Poolife NST Non Stabilized Swimming Pool...,Use only in pools with a skimmer and skimmer b...,NaN,$234.99,1 in stock,Poolife NST Non Stabilized Without Conditioner...,https://bestpoolshop.com/product/poolife-nst-n...,80944,70.2%,29.8%,NaN,NaN,NaN,22422
2,Bulk 50lb 3inch Chlorine Tablets For Swimming ...,Do not allow this product to get damp or wet b...,NaN,$1500.00,2 in stock,Swimming Pool 3 inch Stabilized Chlorine Table...,https://bestpoolshop.com/product/50lb-3inch-ch...,110412,NaN,1%,99%,90%,NaN,NaN
3,CVBR004 Clearview Bromo Bromine Brominating 1i...,"96% Bromine, Slow dissolving, Reduced odor, En...",$71.99,$64.99,Only 8 left in stock,Oreq Clearview Swimming Pool and Spa Bromo Bro...,https://bestpoolshop.com/product/clearview-bro...,119703,NaN,NaN,NaN,NaN,"1-Bromo-3-chloro-5,5-dimethylhydantoin – 96.0%",CVBR004
4,CVTLST005 ClearView Swimming Pool Spa Chlorine...,World’s First Low-Odor Chlorine TabsExperience...,$48.99,$41.99,Only 3 left in stock,CVTLST005 Scent-Trific ClearView Swimming Pool...,https://bestpoolshop.com/product/cvtlst005-cle...,137117,NaN,NaN,NaN,NaN,NaN,CVTLST005


In [83]:
df_2 = pd.read_csv('chlorine_data.csv')
df_2.head()

,title,overview,price,stock,description,link,manufacturer sku,size,brand
0,"AB 3"" Tabs 50 lb | 40631A","Applied Biochemists 3"" Tablets",$284.95,Out of stock,Applied Biochemists® 3” TabletsSlow dissolving...,https://www.freshpoolsupply.com/chemical/chlor...,40631A,50 lb,APPLIED BIO
1,Floating Chlorinator | 58470-110-690,Floating Chemical Dispenser For 3In Tablets,$15.95,In stock,NaN,https://www.freshpoolsupply.com/chemical/chlor...,58470-110-690,NaN,CMP
2,"Scent-Trific 3"" Jumbo Tab 35lb Box of 70 | CVT...","ClearView Scent-Trific Tabs Full Strength, Low...",$359.95,Out of stock,Scent-Trific Jumbo Tabs 3Scent-Trific Chlorine...,https://www.freshpoolsupply.com/chemical/chlor...,CVTLSTCR035,8 Oz,OREQ
3,Shock & Swim 15 1 lb 12pk | CVCF001PK12,Shock & Swim 15 1 lb 12pk,$59.95,Out of stock,Chlor Free Shock &amp; Swim 15Part: CVCF001PK1...,https://www.freshpoolsupply.com/chemical/chlor...,CVCF001PK12,12 lb,OREQ
4,Shock & Swim 15 50 lb | CVCF050,"ClearView Shock & Swim 15, Chlorine Free Shock...",$220.95,In stock,Chlor Free Shock &amp; Swim 15Part: CVCF050Non...,https://www.freshpoolsupply.com/chemical/chlor...,CVCF050,50 lb,OREQ


In [84]:
df_3 = pd.read_csv('epoolsupply_chlorine.csv')
df_3.head()

,title,overview,price,stock,description,link,manufacturer sku,brand
0,"Poolife 3"" Chlorine Tabs (50lbs) | 42118","steady, consistent chlorination | sanitized, c...",$169.95,In stock,"Poolife 3"" Chlorine Tabs arehighly effective c...",https://www.epoolsupply.com/collections/poolif...,42118,Poolife Chlorine & Sanitizers
1,"Poolife 1"" Cleaning Tablets | 42104 (5LBS)",pool maintenance | 1-inch trichlor tablets | k...,$52.99,In stock,"Poolife 1"" Cleaning Tablets arehighly effectiv...",https://www.epoolsupply.com/collections/poolif...,42104,Poolife Chlorine & Sanitizers
2,Poolife Bromine Tablets | 25 LBS (62182),slow-dissolving tablets | continuous disinfect...,$299.00,Out of stock,Key Features:,https://www.epoolsupply.com/collections/poolif...,62182,Poolife Chlorine & Sanitizers
3,Poolife NST Prime Tablets | 4.5 LBS,chlorine | oxidizers | stabilizers | residenti...,$41.99,Out of stock,Poolife NST Prime Tablets are designed forpool...,https://www.epoolsupply.com/collections/poolif...,22427,Poolife Chlorine & Sanitizers
4,Poolife NST Prime Tablets | 9 LBS (22424),slow-dissolving tablets | chlorine sanitizers ...,$72.99,Out of stock,Poolife NST Prime Tablets in the9 LBSsize are ...,https://www.epoolsupply.com/collections/poolif...,22424,Poolife Chlorine & Sanitizers


In [85]:
df_4 = pd.read_csv('poolweb_chlorine.csv')
df_4.head()

,title,overview,price,stock,description,link,product category,poolweb item #,cross reference #,manufacturer,shipping weight
0,Pool Season Chlorinating Tablets Wrapped - 3 I...,3 inch wrapped tablets | 90% available chlorin...,$277.33,209 in stock,Description The Pool Season Chlorinating Table...,https://www.poolweb.com/products/pool-season-c...,Chlorine,HGH-50-3750,47251280,Pool Season,52.0 lb
1,Pool Season Chlorinating Tablets Wrapped - 3 I...,3 inch wrapped tablets | 90% available chlorin...,$147.01,225 in stock,Description The Pool Season Chlorinating Table...,https://www.poolweb.com/products/pool-season-c...,Chlorine,HGH-50-3725,47251270,Pool Season,27.0 lb
2,Pool Season Chlorinating Tablets Wrapped - 3 I...,3 inch wrapped tablets | 90% available chlorin...,$79.82,200 in stock,Description The Pool Season Chlorinating Table...,https://www.poolweb.com/products/pool-season-c...,Chlorine,HGH-50-3708,47251260,Pool Season,10.0 lb
3,Pool Season Non-Chlorine Shock Oxidizer - 1 Lb...,Non-chlorine oxidizer will not raise chlorine ...,$13.85,248 in stock,Description The Pool Season Non-Chlorine Shock...,https://www.poolweb.com/products/pool-season-n...,Chlorine,HGH-50-2849,47251330,Pool Season,2.0 lb
4,Pool Season Chlorinating Concentrate - 50 Lb. ...,Stabilized chlorinating concentrate | Totally ...,$334.29,214 in stock,Description This is a 50 pound bucket of Pool ...,https://www.poolweb.com/products/pool-season-c...,Chlorine,HGH-50-2650,47251320,Pool Season,52.0 lb


In [86]:
df_2.columns

Index(['title', 'overview', 'price', 'stock', 'description', 'link',
       'manufacturer sku', 'size', 'brand'],
      dtype='object')

In [87]:
df_1.head()

,title,overview,price,in_store_price,stock,description,link,product_id,calcium hypochlorite,other ingredients,trichlor-s-triazinetrione,available chlorine,active ingredient,manufacturer sku
0,22421 Poolife NST Non Stabilized Swimming Pool...,Use only in pools with a skimmer and skimmer b...,$185.76,$171.99,Only 7 left in stock,Poolife NST Non Stabilized Without Conditioner...,https://bestpoolshop.com/product/poolife-nst-n...,36140,70.2%,29.8%,NaN,NaN,NaN,22421
1,22422 Poolife NST Non Stabilized Swimming Pool...,Use only in pools with a skimmer and skimmer b...,NaN,$234.99,1 in stock,Poolife NST Non Stabilized Without Conditioner...,https://bestpoolshop.com/product/poolife-nst-n...,80944,70.2%,29.8%,NaN,NaN,NaN,22422
2,Bulk 50lb 3inch Chlorine Tablets For Swimming ...,Do not allow this product to get damp or wet b...,NaN,$1500.00,2 in stock,Swimming Pool 3 inch Stabilized Chlorine Table...,https://bestpoolshop.com/product/50lb-3inch-ch...,110412,NaN,1%,99%,90%,NaN,NaN
3,CVBR004 Clearview Bromo Bromine Brominating 1i...,"96% Bromine, Slow dissolving, Reduced odor, En...",$71.99,$64.99,Only 8 left in stock,Oreq Clearview Swimming Pool and Spa Bromo Bro...,https://bestpoolshop.com/product/clearview-bro...,119703,NaN,NaN,NaN,NaN,"1-Bromo-3-chloro-5,5-dimethylhydantoin – 96.0%",CVBR004
4,CVTLST005 ClearView Swimming Pool Spa Chlorine...,World’s First Low-Odor Chlorine TabsExperience...,$48.99,$41.99,Only 3 left in stock,CVTLST005 Scent-Trific ClearView Swimming Pool...,https://bestpoolshop.com/product/cvtlst005-cle...,137117,NaN,NaN,NaN,NaN,NaN,CVTLST005


In [88]:
def restructure_df(df):
    """
    Keeps only ['title', 'overview', 'price', 'stock', 'description', 'link'] columns.
    Merges any additional columns into 'description' with labels.

    Parameters:
        df (pd.DataFrame): The input DataFrame

    Returns:
        pd.DataFrame: Modified DataFrame with merged description
    """
    
    # Core columns to retain
    final_cols = ['title', 'overview', 'price', 'stock', 'description', 'link']
    
    # Identify extra columns
    extra_cols = [col for col in df.columns if col not in final_cols]

    # Merge extra columns into description
    df['description'] = df.apply(
        lambda row: f"{row['description']} " + 
                    " ".join([f"{col}: {row[col]}" for col in extra_cols if pd.notna(row[col])]), 
        axis=1
    )

    # Keep only final columns
    return df[final_cols]


In [89]:
df_1_new = restructure_df(df_1)
df_2_new = restructure_df(df_2)
df_3_new = restructure_df(df_3)
df_4_new = restructure_df(df_4)

In [90]:
df_1_new.to_csv('bestpoolshop_merged.csv', index=False)
df_2_new.to_csv('freshpool_merged.csv', index=False)
df_3_new.to_csv('epoolsupply_merged.csv', index=False)
df_4_new.to_csv('poolweb_merged.csv', index=False)

In [67]:
df_4_new.head()

,title,overview,price,stock,description,link
0,Pool Season Chlorinating Tablets Wrapped - 3 I...,3 inch wrapped tablets | 90% available chlorin...,$277.33,209 in stock,Description The Pool Season Chlorinating Table...,https://www.poolweb.com/products/pool-season-c...
1,Pool Season Chlorinating Tablets Wrapped - 3 I...,3 inch wrapped tablets | 90% available chlorin...,$147.01,225 in stock,Description The Pool Season Chlorinating Table...,https://www.poolweb.com/products/pool-season-c...
2,Pool Season Chlorinating Tablets Wrapped - 3 I...,3 inch wrapped tablets | 90% available chlorin...,$79.82,200 in stock,Description The Pool Season Chlorinating Table...,https://www.poolweb.com/products/pool-season-c...
3,Pool Season Non-Chlorine Shock Oxidizer - 1 Lb...,Non-chlorine oxidizer will not raise chlorine ...,$13.85,248 in stock,Description The Pool Season Non-Chlorine Shock...,https://www.poolweb.com/products/pool-season-n...
4,Pool Season Chlorinating Concentrate - 50 Lb. ...,Stabilized chlorinating concentrate | Totally ...,$334.29,214 in stock,Description This is a 50 pound bucket of Pool ...,https://www.poolweb.com/products/pool-season-c...


In [80]:
df_1_new['description'][0]

'Poolife NST Non Stabilized Without Conditioner Swimming Pool Chlorine Tablet 20.6lb 22421 in_store_price: $171.99 product_id: 36140 calcium hypochlorite: 70.2% other ingredients: 29.8% manufacturer sku: 22421'

In [68]:
def remove_dollar(price):
    """
    Removes $ symbol, commas, extra spaces from price and returns as float.
    Returns 0.0 if conversion fails.
    """
    try:
        price = str(price).replace("$", "").replace(",", "").strip()
        return float(price)
    except:
        return 0.0


In [69]:
df_1_new.loc[:, 'price'] = df_1_new['price'].apply(remove_dollar)
df_2_new.loc[:, 'price'] = df_2_new['price'].apply(remove_dollar)
df_3_new.loc[:, 'price'] = df_3_new['price'].apply(remove_dollar)
df_4_new.loc[:, 'price'] = df_4_new['price'].apply(remove_dollar)

In [70]:
type(df_1_new['price'][0])

float

In [75]:
import requests
import json
import time

GROQ_API_KEY = "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"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"
MODEL_NAME = "llama-3.3-70b-versatile"

headers = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}

In [76]:
def llm_product_match(prod1, prod2):
    system_prompt = """You are an expert product comparator. 
    Compare two pool-related products and decide if they are the same based on title, overview, and description even if brand differs. 
    Reply only 'Yes' or 'No'."""

    user_prompt = f"""
Product 1:
Title: {prod1.get('title', '')}
Overview: {prod1.get('overview', '')}
Description: {prod1.get('description', '')}

Product 2:
Title: {prod2.get('title', '')}
Overview: {prod2.get('overview', '')}
Description: {prod2.get('description', '')}

Are these products essentially the same? (Yes/No)
"""

    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "max_tokens": 10
    }

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    response = requests.post(GROQ_URL, headers=headers, data=json.dumps(payload))

    if response.status_code != 200:
        print(f"⚠ LLM API Error {response.status_code}: {response.text}")
        return False

    try:
        result = response.json()
        reply = result['choices'][0]['message']['content'].strip().lower()
        return reply.startswith('yes')
    except Exception as e:
        print(f"⚠ JSON Parse Error: {e}, Raw Response: {response.text}")
        return False


In [77]:
def adjust_price_logic(your_price, competitor_price_list):
    """
    Rules:
    - If competitors all have lower prices, set your price 1% below lowest competitor
    - If your price already best, keep it
    - If mixed, adjust to average of lowest two competitor prices minus 1 unit
    """
    valid_prices = [p for p in competitor_price_list if p > 0]

    if not valid_prices:
        return your_price  # No valid competitor price to compare

    min_price = min(valid_prices)
    sorted_prices = sorted(valid_prices)

    # If all competitors cheaper
    if all(p < your_price for p in valid_prices):
        return round(min_price - 1, 2)

    # If mixed, average lowest two and adjust
    if len(sorted_prices) >= 2:
        avg = (sorted_prices[0] + sorted_prices[1]) / 2
        return round(avg - 1, 2)

    # Else keep your price
    return your_price


In [ ]:
df_2_new = df_2_new.copy()
df_2_new['new_price'] = df_2_new['price']
df_2_new['competitor_prices'] = [{} for _ in range(len(df_2_new))]


for idx, row in df_2_new.iterrows():
    
    competitor_price_dict = {}
    matched_prices = []

    for df, comp_name in zip([df_1_new, df_3_new, df_4_new], ["Competitor_1", "Competitor_3", "Competitor_4"]):
        
        for _, comp_row in df.iterrows():
            
            if llm_product_match(row, comp_row):
                price = comp_row.get('price', 0)
                competitor_price_dict[comp_name] = price
                matched_prices.append(price)

                print(f"✅ Matched Product:")
                print(f"→ Site Product: {row['title']} | {row['link']}")
                print(f"→ Competitor: {comp_name} | {comp_row['title']} | {comp_row['link']}")
                print("-" * 80)
            time.sleep(2)
    df_2_new.at[idx, 'competitor_prices'] = competitor_price_dict
    df_2_new.at[idx, 'new_price'] = adjust_price_logic(row['price'], matched_prices)


In [57]:
df_2_new

,title,overview,price,stock,description,link,new_price,competitor_prices
0,"AB 3"" Tabs 50 lb | 40631A","Applied Biochemists 3"" Tablets",284.95,Out of stock,Applied Biochemists® 3” TabletsSlow dissolving...,https://www.freshpoolsupply.com/chemical/chlor...,23.99,"{'Competitor_1': nan, 'Competitor_3': 309.99, ..."
1,Floating Chlorinator | 58470-110-690,Floating Chemical Dispenser For 3In Tablets,15.95,In stock,nan manufacturer sku: 58470-110-690 brand: CMP,https://www.freshpoolsupply.com/chemical/chlor...,15.95,{}
2,"Scent-Trific 3"" Jumbo Tab 35lb Box of 70 | CVT...","ClearView Scent-Trific Tabs Full Strength, Low...",359.95,Out of stock,Scent-Trific Jumbo Tabs 3Scent-Trific Chlorine...,https://www.freshpoolsupply.com/chemical/chlor...,359.95,{}
3,Shock & Swim 15 1 lb 12pk | CVCF001PK12,Shock & Swim 15 1 lb 12pk,59.95,Out of stock,Chlor Free Shock &amp; Swim 15Part: CVCF001PK1...,https://www.freshpoolsupply.com/chemical/chlor...,59.95,{}
4,Shock & Swim 15 50 lb | CVCF050,"ClearView Shock & Swim 15, Chlorine Free Shock...",220.95,In stock,Chlor Free Shock &amp; Swim 15Part: CVCF050Non...,https://www.freshpoolsupply.com/chemical/chlor...,220.95,{}
...,...,...,...,...,...,...,...,...
68,PL TurboShock 1 lb 24pk | 22405PK24,Poolife TurboShock 1 lb 24pk,121.95,Out of stock,Poolife™ TurboShock™ Shock Treatment: Cal Hypo...,https://www.freshpoolsupply.com/chemical/chlor...,121.95,{}
69,NST Floater | 92052,NST Floater,21.95,Out of stock,Poolife™ NST™ Chlorine Floater: Cal Hypo Pool ...,https://www.freshpoolsupply.com/chemical/chlor...,21.95,{}
70,Pink Flamingo Chlorine Dispenser | 32123,Pink Flamingo Chlorine Dispenser,33.95,Out of stock,nan manufacturer sku: 32123 brand: POOLMASTER,https://www.freshpoolsupply.com/chemical/chlor...,33.95,{}
71,Trichlor Granular Ezchlor 50 lb | 12000536,Trichlor Granular Ezchlor 50Lb,216.95,Out of stock,nan manufacturer sku: 12000536 size: 50 lb bra...,https://www.freshpoolsupply.com/chemical/chlor...,216.95,{}
